https://stackoverflow.com/questions/73095228/xarray-cannot-open-url

In [2]:
pip install esgf-pyclient

  Using cached esgf_pyclient-0.3.1-py3-none-any.whl
  Using cached requests_cache-0.9.6-py3-none-any.whl (47 kB)
  Using cached MyProxyClient-2.1.0-py3-none-any.whl (72 kB)
  Using cached url_normalize-1.4.3-py2.py3-none-any.whl (6.8 kB)
  Using cached cattrs-22.2.0-py3-none-any.whl (35 kB)
  Using cached exceptiongroup-1.0.0rc9-py3-none-any.whl (12 kB)
Note: you may need to restart the kernel to use updated packages.


In [34]:
from pyesgf.logon import LogonManager
lm = LogonManager()
lm.is_logged_on()
hostname='esgf-node.llnl.gov'
hostname='esgf-node.ipsl.upmc.fr'
lm.logon(hostname=hostname, interactive=True, bootstrap=True, username='tinaok', password='Clivar-2022')
lm.is_logged_on()

True

In [35]:
from pyesgf.search import SearchConnection
conn = SearchConnection('https://esgf-data.dkrz.de/esg-search', distrib=True)

In [39]:
source_id='CMCC-CM2-HR4'
activity_id='OMIP'
experiment_id='omip2'
variable_id='vmo'
project='CMIP6'
#variant_label='r1i1p1f2',
#data_node='esgf-data3.ceda.ac.uk'

In [40]:
ctx = conn.new_context(facets='*',
    project=project,
    source_id=source_id,
    experiment_id=experiment_id,
    variable=variable_id,
    frequency='mon',
)
ctx.hit_count

1

In [41]:
result = ctx.search()[0]
result.dataset_id

'CMIP6.OMIP.CMCC.CMCC-CM2-HR4.omip2.r1i1p1f1.Omon.vmo.gn.v20200226|esgf-node2.cmcc.it'

In [42]:
files = result.file_context().search()
files[35].urls


-------------------------------------------------------------------------------
Warning - defaulting to search with facets=*

This behavior is kept for backward-compatibility, but ESGF indexes might not
successfully perform a distributed search when this option is used, so some
results may be missing.  For full results, it is recommended to pass a list of
facets of interest when instantiating a context object.  For example,

      ctx = conn.new_context(facets='project,experiment_id')

Only the facets that you specify will be present in the facets_counts dictionary.

This warning is displayed when a distributed search is performed while using the
facets=* default, a maximum of once per context object.  To suppress this warning,
set the environment variable ESGF_PYCLIENT_NO_FACETS_STAR_WARNING to any value
or explicitly use  conn.new_context(facets='*')

-------------------------------------------------------------------------------


defaultdict(list,
            {'HTTPServer': [('http://esgf-node2.cmcc.it/thredds/fileServer/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc',
               'application/netcdf')],
             'GridFTP': [('gsiftp://esgf-node2.cmcc.it:2811//esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc',
               'application/gridftp')],
             'OPENDAP': [('http://esgf-node2.cmcc.it/thredds/dodsC/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc.html',
               'application/opendap-html')],
             'Globus': [('globus:4101e3a0-b7df-11eb-a16a-5fad80e6400b/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc',
               'Globus')]})

In [46]:
gridftp=files[35].urls['GridFTP'][0][0]
http=files[35].urls['HTTPServer'][0][0]
opendap=files[35].urls['OPENDAP'][0][0]
opendap

'http://esgf-node2.cmcc.it/thredds/dodsC/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc.html'

In [ ]:
## opendap access does not work.  (wget does not neither) 

In [45]:
import xarray as xr
ds_agg = xr.open_mfdataset([opendap], engine="pydap")
ds_agg

HTTPError: Timeout

In [ ]:
!wget http://esgf-node2.cmcc.it/thredds/dodsC/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc.html

--2022-10-21 08:49:40--  http://esgf-node2.cmcc.it/thredds/dodsC/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc.html
Resolving esgf-node2.cmcc.it (esgf-node2.cmcc.it)... 193.204.199.223
Connecting to esgf-node2.cmcc.it (esgf-node2.cmcc.it)|193.204.199.223|:80... failed: Connection timed out.
Retrying.

--2022-10-21 08:51:52--  (try: 2)  http://esgf-node2.cmcc.it/thredds/dodsC/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc.html
Connecting to esgf-node2.cmcc.it (esgf-node2.cmcc.it)|193.204.199.223|:80... failed: Connection timed out.
Retrying.

--2022-10-21 08:54:04--  (try: 3)  http://esgf-node2.cmcc.it/thredds/dodsC/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc.html
Connecting to esgf-node2.cmcc.it (esgf-node2.cmcc.it)|193

## http access does not work.  (wget does not work neither) 

In [29]:
import xarray as xr
ds_agg = xr.open_mfdataset([http], concat_dim='time')
ds_agg

Note:Caching=1
Error:curl error: Timeout was reached
curl error details: 


OSError: [Errno -68] NetCDF: I/O failure: b'http://esgf-node2.cmcc.it/thredds/fileServer/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc'

In [30]:
!wget http://esgf-node2.cmcc.it/thredds/dodsC/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc

--2022-10-21 04:51:30--  http://esgf-node2.cmcc.it/thredds/dodsC/esg_dataroot/CMIP6/OMIP/CMCC/CMCC-CM2-HR4/omip2/r1i1p1f1/Omon/vmo/gn/v20200226/vmo_Omon_CMCC-CM2-HR4_omip2_r1i1p1f1_gn_200801-201812.nc
Resolving esgf-node2.cmcc.it (esgf-node2.cmcc.it)... 193.204.199.223
Connecting to esgf-node2.cmcc.it (esgf-node2.cmcc.it)|193.204.199.223|:80... ^C
